<a href="https://colab.research.google.com/github/udothemath/ncku_customer_embedding/blob/main/WenMing/1_data_process_Normal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 如果有使用 coloab 再執行此 cell
from google.colab import drive
drive.mount('/gdrive')
!ln -s /gdrive/MyDrive/colab/NCKU_embedding/data/ ./data

Mounted at /gdrive


In [ ]:
import os
import numpy as np
import pandas as pd
from time import time
from tqdm.notebook import tqdm, trange
from sklearn.preprocessing import MinMaxScaler

In [ ]:
sample_path = './data/sample_50k'

chid_file = 'sample_chid.txt'
chid_dict_file = 'sample_idx_map.npy'
cdtx_file = 'sample_zip_if_cca_cdtx0001_hist.csv'
cust_f_file = 'sample_zip_if_cca_cust_f.csv'
y_file = 'sample_zip_if_cca_y.csv'

# 預下個週期的目標值，如果 shift 為零，代表是預測當下
shift=1


# Load Data
* load the csv that have been preprocessed by [0_sample_file_by_chid](./0_sample_file_by_chid.ipynb)

In [ ]:
chid_array = np.loadtxt(os.path.join(sample_path, chid_file), dtype=np.str)
chid_dict = np.load(os.path.join(sample_path, chid_dict_file), allow_pickle=True).item()
df_cdtx = pd.read_csv(os.path.join(sample_path, cdtx_file)) # 交易記錄檔
df_cust_f = pd.read_csv(os.path.join(sample_path, cust_f_file)) # user feature
df_cust_f.drop_duplicates(ignore_index=True, inplace=True)
df_y = pd.read_csv(os.path.join(sample_path, y_file)) #  預測目標

print(chid_array.shape, len(chid_dict), df_cdtx.shape, df_cust_f.shape, df_y.shape)

(50000,) 50000 (6620047, 10) (1176884, 32) (2882883, 4)


In [ ]:
df_cdtx.head(2)

,bnsfg,bnspt,chid,csmdt,iterm,mcc,objam,scity,tcode,hcefg
0,N,0,+10kXREN0NI0BQgqeDnT4g==,2018-01-01,0,5411,915,TAICHUNG,5,NaN
1,N,0,+1cvhSoBR5xt9eHv3bVU8g==,2018-01-01,0,MRT5,2600,NaN,5,NaN


In [ ]:
df_cust_f.head(2)

,chid,data_ym,monin,wrky,first_mob,data_dt,masts,educd,naty,trdtp,...,constant_u2_ind,constant_u3_ind,constant_u4_ind,constant_l2_ind,constant_l3_ind,constant_l4_ind,constant_change,growth_rate,monotone_up,monotone_down
0,++03Yg+R1oaKBt0f2gWCgA==,2017-12-01,211645,0,80.0,2018-01-01,ea9d465d7361343a138603660b263e4f9fdb7bc04c5843...,6ae4ba148e6b8652e0768cf3db4bbade8ffc50c94d08a5...,7fcfcd907e0d490a37d0e7df45db65bc6bf009c3d90f66...,f0243f3eded4a032b3ad01b034d5d6a96b92d8ba6f7344...,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.8,0.0,1.0
1,++03Yg+R1oaKBt0f2gWCgA==,2018-01-01,211645,0,81.0,2018-02-01,ea9d465d7361343a138603660b263e4f9fdb7bc04c5843...,6ae4ba148e6b8652e0768cf3db4bbade8ffc50c94d08a5...,7fcfcd907e0d490a37d0e7df45db65bc6bf009c3d90f66...,f0243f3eded4a032b3ad01b034d5d6a96b92d8ba6f7344...,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.8,0.0,3.0


In [ ]:
df_y.head(2)

,data_dt,chid,stonc_tag,y
0,2018-01-01,++09R1U4V8zedf5TUopHeQ==,D1,14737
1,2018-01-01,++4jH9yOMLHOVNVM4oP8IA==,R2,27274


## Dobule verify and do filtering

In [ ]:
df_cdtx = df_cdtx[df_cdtx.chid.isin(chid_array)].copy()
df_cust_f = df_cust_f[df_cust_f.chid.isin(chid_array)].copy()
df_y = df_y[df_y.chid.isin(chid_array)].copy()

## Replace user id with serial id

In [ ]:
df_cdtx.chid = df_cdtx.chid.map(chid_dict)
df_cust_f.chid = df_cust_f.chid.map(chid_dict)
df_y.chid = df_y.chid.map(chid_dict)

In [ ]:
df_cust_f.head(1)

,chid,data_ym,monin,wrky,first_mob,data_dt,masts,educd,naty,trdtp,...,constant_u2_ind,constant_u3_ind,constant_u4_ind,constant_l2_ind,constant_l3_ind,constant_l4_ind,constant_change,growth_rate,monotone_up,monotone_down
0,41654,2017-12-01,211645,0,80.0,2018-01-01,ea9d465d7361343a138603660b263e4f9fdb7bc04c5843...,6ae4ba148e6b8652e0768cf3db4bbade8ffc50c94d08a5...,7fcfcd907e0d490a37d0e7df45db65bc6bf009c3d90f66...,f0243f3eded4a032b3ad01b034d5d6a96b92d8ba6f7344...,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.8,0.0,1.0


## 統一將 month 的啟始日設為 1 號

In [ ]:
df_cdtx['month'] = df_cdtx.csmdt.apply(lambda x: x[:-3]+'-01')
df_cdtx.tail(2)

,bnsfg,bnspt,chid,csmdt,iterm,mcc,objam,scity,tcode,hcefg,month
6620045,N,0,23761,2020-01-31,0,5542,1347,YUNLIN,5,NaN,2020-01-01
6620046,N,0,23761,2020-01-31,0,5542,806,YUNLIN,5,NaN,2020-01-01


In [ ]:
## 填滿後12個月

list_chid = sorted(df_y.chid.unique())
list_month = sorted(df_y.data_dt.unique())[12:]

# 先產出一個空的 dataFrame
df_full_y_sum = pd.DataFrame({
    'chid': list_chid*len(list_month),
}).sort_values(by='chid', ignore_index=True)
df_full_y_sum['data_dt'] = list_month*len(list_chid)

print(df_full_y_sum.shape)
df_full_y_sum.head(3)

(600000, 2)


,chid,data_dt
0,0,2019-01-01
1,0,2019-02-01
2,0,2019-03-01


In [ ]:
## join feature ['婚姻狀態','教育',  '國籍', '行業別',  '職稱' , '客戶來源']
category_cols = ['masts', 'educd', 'naty', 'trdtp', 'poscd', 'cuorg']

numeric_cols = sorted(set(df_cust_f.columns) - set(category_cols) - set(['chid', 'data_ym', 'data_dt']), 
                      key=list(df_cust_f.columns).index)

In [ ]:
df_cust_f.head(2)

,chid,data_ym,monin,wrky,first_mob,data_dt,masts,educd,naty,trdtp,poscd,cuorg,cycam,slam,sum_area_c,sum_u2_ind,sum_u3_ind,sum_u4_ind,sum_l2_ind,sum_l3_ind,sum_l4_ind,constant_area_c,constant_u2_ind,constant_u3_ind,constant_u4_ind,constant_l2_ind,constant_l3_ind,constant_l4_ind,constant_change,growth_rate,monotone_up,monotone_down
0,3801,2017-12-01,229533,0,27.0,2018-01-01,ea9d465d7361343a138603660b263e4f9fdb7bc04c5843...,a2822b0ed756338d1ff31695467786db874a02189bdbcf...,7fcfcd907e0d490a37d0e7df45db65bc6bf009c3d90f66...,53f490495a0ec0ee6aaf9c03252e5989b3624d312c7b72...,9484866f7342a8442f275ab5cf94c4be10c4474bb036ca...,3f0b90c5be164157c85dc759cbdfe753d251e6997e6019...,520000,520000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.2,1.0,0.0
1,3801,2018-01-01,229533,0,28.0,2018-02-01,ea9d465d7361343a138603660b263e4f9fdb7bc04c5843...,a2822b0ed756338d1ff31695467786db874a02189bdbcf...,7fcfcd907e0d490a37d0e7df45db65bc6bf009c3d90f66...,53f490495a0ec0ee6aaf9c03252e5989b3624d312c7b72...,9484866f7342a8442f275ab5cf94c4be10c4474bb036ca...,3f0b90c5be164157c85dc759cbdfe753d251e6997e6019...,520000,520000,6.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0


In [ ]:
df_full_y_sum = df_full_y_sum.merge(df_cust_f[['chid', 'data_ym'] + category_cols + numeric_cols], 
                                    how='left', 
                                    left_on=['chid', 'data_dt'], 
                                    right_on=['chid', 'data_ym'])

In [ ]:
df_full_y_sum.head(20)[10:15]

,chid,data_dt,data_ym,masts,educd,naty,trdtp,poscd,cuorg,monin,wrky,first_mob,cycam,slam,sum_area_c,sum_u2_ind,sum_u3_ind,sum_u4_ind,sum_l2_ind,sum_l3_ind,sum_l4_ind,constant_area_c,constant_u2_ind,constant_u3_ind,constant_u4_ind,constant_l2_ind,constant_l3_ind,constant_l4_ind,constant_change,growth_rate,monotone_up,monotone_down
10,0,2019-11-01,2019-11-01,5a7095e93ba2ee05247bba34dd4509a51e2798d33a1c7c...,6ae4ba148e6b8652e0768cf3db4bbade8ffc50c94d08a5...,7fcfcd907e0d490a37d0e7df45db65bc6bf009c3d90f66...,3855c51a8c67d0aa80280f5dd51cf7ae91e98886d13f09...,9484866f7342a8442f275ab5cf94c4be10c4474bb036ca...,3f0b90c5be164157c85dc759cbdfe753d251e6997e6019...,119803.0,52.0,92.0,156000.0,202800.0,1.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.8,0.0,6.0
11,0,2019-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,1,2019-01-01,2019-01-01,5a7095e93ba2ee05247bba34dd4509a51e2798d33a1c7c...,a2822b0ed756338d1ff31695467786db874a02189bdbcf...,7fcfcd907e0d490a37d0e7df45db65bc6bf009c3d90f66...,4e55935aee7e67e2892b3a22b4d5a21ab77ed239947e58...,5c5e9c182ca75b1747c92227c6b1e8d2eeb1f4d0e205ae...,3f0b90c5be164157c85dc759cbdfe753d251e6997e6019...,289541.0,73.0,159.0,1040000.0,1352000.0,4.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.8,0.0,1.0
13,1,2019-02-01,2019-02-01,5a7095e93ba2ee05247bba34dd4509a51e2798d33a1c7c...,a2822b0ed756338d1ff31695467786db874a02189bdbcf...,7fcfcd907e0d490a37d0e7df45db65bc6bf009c3d90f66...,4e55935aee7e67e2892b3a22b4d5a21ab77ed239947e58...,5c5e9c182ca75b1747c92227c6b1e8d2eeb1f4d0e205ae...,3f0b90c5be164157c85dc759cbdfe753d251e6997e6019...,289541.0,73.0,160.0,1040000.0,1352000.0,1.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.8,0.0,3.0
14,1,2019-03-01,2019-03-01,5a7095e93ba2ee05247bba34dd4509a51e2798d33a1c7c...,a2822b0ed756338d1ff31695467786db874a02189bdbcf...,7fcfcd907e0d490a37d0e7df45db65bc6bf009c3d90f66...,4e55935aee7e67e2892b3a22b4d5a21ab77ed239947e58...,5c5e9c182ca75b1747c92227c6b1e8d2eeb1f4d0e205ae...,3f0b90c5be164157c85dc759cbdfe753d251e6997e6019...,289541.0,73.0,161.0,1040000.0,1352000.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.9,0.0,6.0


## fill na value, numerical: 0, category: '-1'


In [ ]:
#df_full_y_sum.dropna(thresh=len(numeric_cols+category_cols), inplace=True)

values = dict()

for col in numeric_cols:
    values[col] = 0
    
for col in category_cols:
    values[col] = '-1'

# 真的特定的值來取代成 N/A , 
df_full_y_sum.fillna(value=values, inplace=True)
print(df_full_y_sum.shape)


(600000, 32)


In [ ]:
df_full_y_sum.head(20)[10:15]

,chid,data_dt,data_ym,masts,educd,naty,trdtp,poscd,cuorg,monin,wrky,first_mob,cycam,slam,sum_area_c,sum_u2_ind,sum_u3_ind,sum_u4_ind,sum_l2_ind,sum_l3_ind,sum_l4_ind,constant_area_c,constant_u2_ind,constant_u3_ind,constant_u4_ind,constant_l2_ind,constant_l3_ind,constant_l4_ind,constant_change,growth_rate,monotone_up,monotone_down
10,0,2019-11-01,2019-11-01,5a7095e93ba2ee05247bba34dd4509a51e2798d33a1c7c...,6ae4ba148e6b8652e0768cf3db4bbade8ffc50c94d08a5...,7fcfcd907e0d490a37d0e7df45db65bc6bf009c3d90f66...,3855c51a8c67d0aa80280f5dd51cf7ae91e98886d13f09...,9484866f7342a8442f275ab5cf94c4be10c4474bb036ca...,3f0b90c5be164157c85dc759cbdfe753d251e6997e6019...,119803.0,52.0,92.0,156000.0,202800.0,1.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.8,0.0,6.0
11,0,2019-12-01,NaN,-1,-1,-1,-1,-1,-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,1,2019-01-01,2019-01-01,5a7095e93ba2ee05247bba34dd4509a51e2798d33a1c7c...,a2822b0ed756338d1ff31695467786db874a02189bdbcf...,7fcfcd907e0d490a37d0e7df45db65bc6bf009c3d90f66...,4e55935aee7e67e2892b3a22b4d5a21ab77ed239947e58...,5c5e9c182ca75b1747c92227c6b1e8d2eeb1f4d0e205ae...,3f0b90c5be164157c85dc759cbdfe753d251e6997e6019...,289541.0,73.0,159.0,1040000.0,1352000.0,4.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.8,0.0,1.0
13,1,2019-02-01,2019-02-01,5a7095e93ba2ee05247bba34dd4509a51e2798d33a1c7c...,a2822b0ed756338d1ff31695467786db874a02189bdbcf...,7fcfcd907e0d490a37d0e7df45db65bc6bf009c3d90f66...,4e55935aee7e67e2892b3a22b4d5a21ab77ed239947e58...,5c5e9c182ca75b1747c92227c6b1e8d2eeb1f4d0e205ae...,3f0b90c5be164157c85dc759cbdfe753d251e6997e6019...,289541.0,73.0,160.0,1040000.0,1352000.0,1.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.8,0.0,3.0
14,1,2019-03-01,2019-03-01,5a7095e93ba2ee05247bba34dd4509a51e2798d33a1c7c...,a2822b0ed756338d1ff31695467786db874a02189bdbcf...,7fcfcd907e0d490a37d0e7df45db65bc6bf009c3d90f66...,4e55935aee7e67e2892b3a22b4d5a21ab77ed239947e58...,5c5e9c182ca75b1747c92227c6b1e8d2eeb1f4d0e205ae...,3f0b90c5be164157c85dc759cbdfe753d251e6997e6019...,289541.0,73.0,161.0,1040000.0,1352000.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.9,0.0,6.0


## GroupBy 出 Feature
* e.q. sum of the consumption group by chid&month
* temp_cdtx.index 因為 group by 而產出 index

In [ ]:
## 取得整個月的 objam 
temp_cdtx = df_cdtx.groupby(['chid', 'month']).sum()
df_cdtx_objam = pd.DataFrame(list(map(list, temp_cdtx.index)), columns=['chid', 'data_dt'])
df_cdtx_objam['objam'] = temp_cdtx.objam.values

### 取得整個月的 y
#temp_y = df_y.groupby(['chid', 'data_dt']).sum()
#df_y_sum = pd.DataFrame(list(map(list, temp_y.index)), columns=['chid', 'data_dt'])
#df_y_sum['y'] = temp_y['y'].values

In [ ]:
df_cdtx_objam.head(1)

,chid,data_dt,objam
0,0,2018-01-01,37799


## Merge Objective Value
* merge from df_y_sum

In [ ]:
## join objam & y 

df_full_y_sum = df_full_y_sum.merge(df_cdtx_objam, 
                                    how='left', 
                                    left_on=['chid', 'data_dt'], 
                                    right_on=['chid', 'data_dt']).fillna(0)

#df_full_y_sum = df_full_y_sum.merge(df_y_sum, 
#                                    how='left', 
#                                    left_on=['chid', 'data_dt'], 
#                                    right_on=['chid', 'data_dt']).fillna(0)
df_full_y_sum.head(5)

,chid,data_dt,data_ym,masts,educd,naty,trdtp,poscd,cuorg,monin,wrky,first_mob,cycam,slam,sum_area_c,sum_u2_ind,sum_u3_ind,sum_u4_ind,sum_l2_ind,sum_l3_ind,sum_l4_ind,constant_area_c,constant_u2_ind,constant_u3_ind,constant_u4_ind,constant_l2_ind,constant_l3_ind,constant_l4_ind,constant_change,growth_rate,monotone_up,monotone_down,objam
0,0,2019-01-01,2019-01-01,5a7095e93ba2ee05247bba34dd4509a51e2798d33a1c7c...,6ae4ba148e6b8652e0768cf3db4bbade8ffc50c94d08a5...,7fcfcd907e0d490a37d0e7df45db65bc6bf009c3d90f66...,3855c51a8c67d0aa80280f5dd51cf7ae91e98886d13f09...,9484866f7342a8442f275ab5cf94c4be10c4474bb036ca...,3f0b90c5be164157c85dc759cbdfe753d251e6997e6019...,119803.0,52.0,82.0,156000.0,202800.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,0.0,17899.0
1,0,2019-02-01,2019-02-01,5a7095e93ba2ee05247bba34dd4509a51e2798d33a1c7c...,6ae4ba148e6b8652e0768cf3db4bbade8ffc50c94d08a5...,7fcfcd907e0d490a37d0e7df45db65bc6bf009c3d90f66...,3855c51a8c67d0aa80280f5dd51cf7ae91e98886d13f09...,9484866f7342a8442f275ab5cf94c4be10c4474bb036ca...,3f0b90c5be164157c85dc759cbdfe753d251e6997e6019...,119803.0,52.0,83.0,156000.0,202800.0,0.0,1.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.9,0.0,1.0,33520.0
2,0,2019-03-01,2019-03-01,5a7095e93ba2ee05247bba34dd4509a51e2798d33a1c7c...,6ae4ba148e6b8652e0768cf3db4bbade8ffc50c94d08a5...,7fcfcd907e0d490a37d0e7df45db65bc6bf009c3d90f66...,3855c51a8c67d0aa80280f5dd51cf7ae91e98886d13f09...,9484866f7342a8442f275ab5cf94c4be10c4474bb036ca...,3f0b90c5be164157c85dc759cbdfe753d251e6997e6019...,119803.0,52.0,84.0,156000.0,202800.0,1.0,1.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.8,0.0,1.0,30540.0
3,0,2019-04-01,2019-04-01,5a7095e93ba2ee05247bba34dd4509a51e2798d33a1c7c...,6ae4ba148e6b8652e0768cf3db4bbade8ffc50c94d08a5...,7fcfcd907e0d490a37d0e7df45db65bc6bf009c3d90f66...,3855c51a8c67d0aa80280f5dd51cf7ae91e98886d13f09...,9484866f7342a8442f275ab5cf94c4be10c4474bb036ca...,3f0b90c5be164157c85dc759cbdfe753d251e6997e6019...,119803.0,52.0,85.0,156000.0,202800.0,0.0,2.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.9,0.0,1.0,206918.0
4,0,2019-05-01,2019-05-01,5a7095e93ba2ee05247bba34dd4509a51e2798d33a1c7c...,6ae4ba148e6b8652e0768cf3db4bbade8ffc50c94d08a5...,7fcfcd907e0d490a37d0e7df45db65bc6bf009c3d90f66...,3855c51a8c67d0aa80280f5dd51cf7ae91e98886d13f09...,9484866f7342a8442f275ab5cf94c4be10c4474bb036ca...,3f0b90c5be164157c85dc759cbdfe753d251e6997e6019...,119803.0,52.0,86.0,156000.0,202800.0,2.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.8,0.0,3.0,10753.0


## Encode category value

In [ ]:
mapper = {col: {value: index for index, value in enumerate(sorted(df_full_y_sum[col].unique()))} 
          for col in category_cols}

## 進行轉換

In [ ]:
df_full_y_sum[category_cols] = df_full_y_sum[category_cols].apply(lambda x: x.map(mapper[x.name]))

print(df_full_y_sum.shape)
df_full_y_sum.head(2)

(600000, 33)


,chid,data_dt,data_ym,masts,educd,naty,trdtp,poscd,cuorg,monin,wrky,first_mob,cycam,slam,sum_area_c,sum_u2_ind,sum_u3_ind,sum_u4_ind,sum_l2_ind,sum_l3_ind,sum_l4_ind,constant_area_c,constant_u2_ind,constant_u3_ind,constant_u4_ind,constant_l2_ind,constant_l3_ind,constant_l4_ind,constant_change,growth_rate,monotone_up,monotone_down,objam
0,0,2019-01-01,2019-01-01,1,2,2,4,6,8,119803.0,52.0,82.0,156000.0,202800.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,0.0,17899.0
1,0,2019-02-01,2019-02-01,1,2,2,4,6,8,119803.0,52.0,83.0,156000.0,202800.0,0.0,1.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.9,0.0,1.0,33520.0


In [ ]:
df_full_y_sum.drop(columns=['data_ym'], inplace=True)

ignore_cols = ['data_dt']
category_cols = ['chid'] + category_cols
numeric_cols = sorted(set(df_full_y_sum.columns) - set(category_cols) - set(ignore_cols), 
                      key=list(df_full_y_sum.columns).index)

print(len(ignore_cols), ignore_cols)
print(len(category_cols), category_cols)
print(len(numeric_cols), numeric_cols)

1 ['data_dt']
7 ['chid', 'masts', 'educd', 'naty', 'trdtp', 'poscd', 'cuorg']
24 ['monin', 'wrky', 'first_mob', 'cycam', 'slam', 'sum_area_c', 'sum_u2_ind', 'sum_u3_ind', 'sum_u4_ind', 'sum_l2_ind', 'sum_l3_ind', 'sum_l4_ind', 'constant_area_c', 'constant_u2_ind', 'constant_u3_ind', 'constant_u4_ind', 'constant_l2_ind', 'constant_l3_ind', 'constant_l4_ind', 'constant_change', 'growth_rate', 'monotone_up', 'monotone_down', 'objam']


In [ ]:
df_full_y_sum.head(1)

,chid,data_dt,masts,educd,naty,trdtp,poscd,cuorg,monin,wrky,first_mob,cycam,slam,sum_area_c,sum_u2_ind,sum_u3_ind,sum_u4_ind,sum_l2_ind,sum_l3_ind,sum_l4_ind,constant_area_c,constant_u2_ind,constant_u3_ind,constant_u4_ind,constant_l2_ind,constant_l3_ind,constant_l4_ind,constant_change,growth_rate,monotone_up,monotone_down,objam
0,0,2019-01-01,1,2,2,4,6,8,119803.0,52.0,82.0,156000.0,202800.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,0.0,17899.0


### 產出預測目標

In [ ]:
def data_split(df, numeric_cols=[], category_cols=[], test_size=0.166):
    
    df = df[category_cols + numeric_cols].copy()
    x_train, x_test, y_train, y_test = [], [], [], []
            
    for i in tqdm(sorted(df.chid.unique())):
        data = df[df.chid == i]
        last = data.shape[0] - 1
        test_num = round(data.shape[0]*test_size)            

        x_train.append(data.iloc[0:last - test_num])
        y_train.append(data.iloc[shift:last - test_num + shift, [-1]])

        x_test.append(data.iloc[last - test_num: last])
        y_test.append(data.iloc[last - test_num + shift: last + shift, [-1]])

    x_train = pd.concat(x_train)
    y_train = pd.concat(y_train)
    
    x_test = pd.concat(x_test)
    y_test = pd.concat(y_test)
    
    return x_train, x_test, y_train, y_test

In [ ]:
df_full_y_sum[df_full_y_sum.chid == 0].iloc[0:10]

,chid,data_dt,masts,educd,naty,trdtp,poscd,cuorg,monin,wrky,first_mob,cycam,slam,sum_area_c,sum_u2_ind,sum_u3_ind,sum_u4_ind,sum_l2_ind,sum_l3_ind,sum_l4_ind,constant_area_c,constant_u2_ind,constant_u3_ind,constant_u4_ind,constant_l2_ind,constant_l3_ind,constant_l4_ind,constant_change,growth_rate,monotone_up,monotone_down,objam
0,0,2019-01-01,1,2,2,4,6,8,119803.0,52.0,82.0,156000.0,202800.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,0.0,17899.0
1,0,2019-02-01,1,2,2,4,6,8,119803.0,52.0,83.0,156000.0,202800.0,0.0,1.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.9,0.0,1.0,33520.0
2,0,2019-03-01,1,2,2,4,6,8,119803.0,52.0,84.0,156000.0,202800.0,1.0,1.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.8,0.0,1.0,30540.0
3,0,2019-04-01,1,2,2,4,6,8,119803.0,52.0,85.0,156000.0,202800.0,0.0,2.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.9,0.0,1.0,206918.0
4,0,2019-05-01,1,2,2,4,6,8,119803.0,52.0,86.0,156000.0,202800.0,2.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.8,0.0,3.0,10753.0
5,0,2019-06-01,1,2,2,4,6,8,119803.0,52.0,87.0,156000.0,202800.0,0.0,1.0,1.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9,0.0,5.0,6058.0
6,0,2019-07-01,1,2,2,4,6,8,119803.0,52.0,88.0,156000.0,202800.0,4.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9,0.0,1.0,135653.0
7,0,2019-08-01,1,2,2,4,6,8,119803.0,52.0,89.0,156000.0,202800.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.9,0.0,3.0,39688.0
8,0,2019-09-01,1,2,2,4,6,8,119803.0,52.0,90.0,156000.0,202800.0,5.0,1.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.8,0.0,3.0,2600.0
9,0,2019-10-01,1,2,2,4,6,8,119803.0,52.0,91.0,156000.0,202800.0,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.8,0.0,5.0,7358.0


In [ ]:
x_train, x_test, y_train, y_test = data_split(df_full_y_sum, numeric_cols, category_cols, test_size=0.166)    

num_chid = len(set(df_full_y_sum.chid))
print('train:{}, test:{}'.format(x_train.shape[0]//num_chid, x_test.shape[0]//num_chid))
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)


train:9, test:2
(450000, 31) (450000, 1) (100000, 31) (100000, 1)


In [ ]:
x_test.head()

,chid,masts,educd,naty,trdtp,poscd,cuorg,monin,wrky,first_mob,...,constant_u3_ind,constant_u4_ind,constant_l2_ind,constant_l3_ind,constant_l4_ind,constant_change,growth_rate,monotone_up,monotone_down,objam
9,0,1,5,2,23,5,7,347563.0,0.0,74.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.4,2.0,0.0,223086.0
10,0,1,5,2,23,5,7,347563.0,0.0,75.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.8,0.0,2.0,9776.0
21,1,1,5,2,7,6,7,300695.0,0.0,293.0,...,0.0,0.0,0.0,0.0,0.0,2.0,1.2,1.0,0.0,245253.0
22,1,1,5,2,7,6,7,300695.0,0.0,294.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.1,3.0,0.0,200128.0
33,2,3,5,2,3,1,7,319056.0,0.0,126.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.9,0.0,1.0,2444.0


In [ ]:
y_test.head()

,objam
10,9776.0
11,430831.0
22,200128.0
23,121966.0
34,9189.0


In [ ]:
os.makedirs(os.path.join(sample_path, 'Normal_%d'%shift))

In [ ]:
x_train.to_csv(os.path.join(sample_path, 'Normal_%d'%shift, 'x_train.csv'), index=False, encoding='utf-8')
x_test.to_csv(os.path.join(sample_path, 'Normal_%d'%shift, 'x_test.csv'), index=False, encoding='utf-8')
y_train.to_csv(os.path.join(sample_path, 'Normal_%d'%shift, 'y_train.csv'), index=False, encoding='utf-8')
y_test.to_csv(os.path.join(sample_path, 'Normal_%d'%shift, 'y_test.csv'), index=False, encoding='utf-8')

In [ ]:
pd.read_csv("./data/sample_50k/Normal_%d/x_train.csv"%shift)[['chid','objam']].head(20)

,chid,objam
0,0,17899.0
1,0,33520.0
2,0,30540.0
3,0,206918.0
4,0,10753.0
5,0,6058.0
6,0,135653.0
7,0,39688.0
8,0,2600.0
9,1,204880.0


In [ ]:
pd.read_csv("./data/sample_50k/Normal_%d/y_train.csv"%shift).head(3)

,objam
0,33520.0
1,30540.0
2,206918.0


In [ ]:
np.save(os.path.join(sample_path, 'Normal_%d'%shift, 'feature_map'), mapper)
np.save(os.path.join(sample_path, 'Normal_%d'%shift, 'feature'), {
    'category_columns':category_cols, 
    'numeric_columns':numeric_cols
})